# Experimental Summary

This script has been developed to measure local strain ε defined as ε = ΔL/L0 in a FCC steel sample under elastic strain in a stress rig. Measured at V20, HZB, Berlin, September 2018 by Peter Kadletz.

λ = 2dsinθ, where 2θ = π (transmission), edges characterise the Bragg condition and hence λ = 2d. Therefore strain is easily computed from the wavelength measurement of of a Bragg edge directly, using un-loaded vs loaded experimental runs (and reference mesurements). The known Miller indices of the crystal structure (FCC) are used to predict the wavelength where the Bragg edges should exist, which is bound by the reachable wavelength extents for the instrument. This provides an approximate region to apply a fit.  A complement error function is used to fit each Bragg edge, and a refined centre location (λ) for the edge is used in the strain measurement. Because each bragg edge can be identified individually, one can determine anisotropic strain across the unit cell in the reachable crystallographic directions. In addition the image processing allows for spacial grouping so localised effects, such as those on unconstrained edges of the sample or in necking regions of the sample can be treated seperately. The plotted outputs in the script aim to capture this.

<img src="IMG_2290.jpg" width="500"/>

# Setup

In [ ]:
import scipp as sc
import numpy as np
from scipp.plot import plot
import matplotlib.pyplot as plt
import utils
from utils import wfm

# Reduction

## Load the data files and instrument geometry

In [ ]:
ds = utils.load()

In [ ]:
ds

## Raw data visualization

In [ ]:
plot(ds["sample"])

## Converting time coordinate to TOF

Use the instrument geometry and chopper cascade parameters to compute time-distance diagram.

In [ ]:
plt.ion()
v20setup = wfm.v20.setup()
# Feed in the detector position to the instrument chopper cascade
v20setup['info']['detector_positions'] = {
    "GP2": -ds.coords['source-position'].value[2] + v20setup['info']['wfm_choppers_midpoint'] + sc.geometry.z(
    ds.coords["position"])['x', 0]['y', 0].value}

frames = wfm.get_frames(instrument=v20setup, plot=True)
frames

We overlay the frame locations onto a spectrum of integrated counts

In [ ]:
plt.ioff()
fig1, ax1 = plt.subplots()
plot(sc.sum(sc.sum(ds["reference"], 'x'), 'y'), ax=ax1)
for i in range(len(frames["GP2"]["left_edges"])):
    ax1.axvspan(frames["GP2"]["left_edges"][i], frames["GP2"]["right_edges"][i], color="C{}".format(i), alpha=0.3)
fig1.canvas.draw_idle()
fig1.canvas

Extract the sections in the original data using value-based slicing and shift the coordinates:

In [ ]:
plt.ion()
stitched = wfm.stitch(data=ds, dim="t", frames=frames["GP2"], plot=True)
stitched

To stitch the data, we make a common container with a TOF axis spanning the entire range, and sum the counts from the different frames.

In [ ]:
plt.ioff()
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Crop to relevant Tof section

In [ ]:
tof_start = 9.0e3*sc.units.us
tof_end = 2.75e4*sc.units.us
stitched = stitched["tof", tof_start:tof_end].copy()
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Transmission Masking

Divides the integrated sample counts with an open beam reference. Any values > masking threshold will be masked. The adj pixels step checks for random pixels which were left unmasked or masked with all their neighbours having the same mask value. These are forced to True or false depending on their neighbour value.

In [ ]:
integrated = sc.sum(stitched, 'tof')
integrated /= integrated["reference"]

In [ ]:
masking_threshold = 0.80 * sc.units.one

for key in ["sample", "sample_elastic"]:
    mask = sc.greater(integrated[key].data, masking_threshold)
    # Apply some neighbour smoothing to the masks
    mask = utils.operations.mask_from_adj_pixels(mask)
    stitched[key].masks["non-sample-region"] = mask
stitched

In [ ]:
plot(sc.sum(stitched["sample"], "tof"))

## Convert to wavelength

In [ ]:
wavelength = sc.neutron.convert(stitched, "tof", "wavelength")

# Rebin to common wavelength axis
edges = sc.array(dims=["wavelength"],
                       values=np.linspace(2.0, 5.5, 129), unit=sc.units.angstrom)
wavelength = sc.rebin(wavelength, "wavelength", edges)

wavelength

## Apply mean filter

In [ ]:
for key in wavelength:
    wavelength[key].data = utils.operations.mean_from_adj_pixels(wavelength[key].data)

In [ ]:
plot(wavelength["sample"])

## Other visualizations

### Show difference between sample and elastic

In [ ]:
plot(wavelength["sample"] - wavelength["sample_elastic"])

### $-\ln(I / I_0)$ plot

In [ ]:
plot(-sc.log(sc.sum(wavelength["sample_elastic"], "wavelength") / 
             sc.sum(wavelength["reference"], "wavelength")),
     vmin=0.25, vmax=0.33)

## Group detector pixels to increase signal to noise

In [ ]:
grouped = utils.groupby2D(wavelength, nbins=27)

In [ ]:
plot(grouped["sample"])

In [ ]:
# plot({"original": sc.sum(sc.sum(wavelength, 'x'), 'y'),
#       "grouped": sc.sum(sc.sum(grouped, 'x'), 'y')}, grid=True)

In [ ]:
# plot(sc.sum(sc.sum(rebinned, 'x'), 'y'), grid=True)

In [ ]:
for key, item in grouped.items():
    item.masks["zero-counts"] = item.data == 0.0*sc.units.counts
grouped

## Normalization

In [ ]:
# Normalize by open beam
normalized = grouped / grouped["reference"]
del normalized["reference"]

In [ ]:
replacement = sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan": replacement, "posinf": replacement, "neginf": replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

plot(sc.sum(sc.sum(normalized, 'x'), 'y'), grid=True)

# Bragg edge fitting

We will first carry out Bragg-edge fitting on the sum of all counts in the sample.

In [ ]:
sample_summed = sc.sum(sc.sum(normalized["sample"], 'x'), 'y')
plot(sample_summed)

## Calculate expected Bragg edge positions 

In [ ]:
# Bragg edge position, in Angstrom, taken from COD entry 9008469
FCC_a = 3.5

# These miller indices for the given unit cell yield bragg edges
# between the maximum and minimum wavelength range.
indices_FCC = [(1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)]

Bragg_edges_FCC = utils.create_Braggedge_list(FCC_a, indices_FCC)

## Run the fitting (currently using Mantid under the hood)

In [ ]:
fit_params = utils.fit(Bragg_edges_FCC, reference=sample_summed)

In [ ]:
import scipy as sp
fig, ax = plt.subplots()
sample_summed.plot(ax=ax)
# x = np.linspace(2.0, 5.5, 200)
# ax.plot(x, (270 - 10.0*x))
for i, f in enumerate(fit_params):
    x = np.linspace(f['f1.x0'].value - 0.3, f['f1.x0'].value + 0.3, 100)
    y=f['f1.h'].value*sp.special.erf(f['f1.a'].value*(x-f['f1.x0'].value)) + (f['f0.A0'].value + f['f0.A1'].value*x)
    ax.plot(x, y, color="C{}".format(i+1), label=Bragg_edges_FCC.coords['miller-indices'].values[i])
ax.legend()
fig.canvas